In [1]:
import scipy as sp
from pyscf import gto, dft, lib, mp, cc, scf, tools

In [2]:
import numpy as np
import sys
sys.path.append( './src/' )

In [3]:
matrix_dot = lambda A, B: np.einsum('ij,ij', A, B)

In [4]:
geometry = [
['O@1', (1.2322305822, -0.2731895077, -0.1276123902)],
['H@1', (1.2473876659, -0.8998737590, 0.6150681570)],
    
['C#', (-1.2129704155, -0.2295285634,-0.0097156258)],
['H#', (-2.0801425360, 0.4329727646,0.0722817289)],
['H#', (-1.2655910941, -0.9539857247, 0.8097953440)],
    
['C#', (0.0849758188, 0.5590385475, 0.0510545434)],
['H#', (0.1506137362, 1.1200249874, 0.9943015309)],
['H#', (0.1316093068, 1.2841805400, -0.7645223601)],
['H#', (-1.2737541560, -0.7748626513, -0.9540587845)]
]

In [5]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(*geometry[0][1:][0], marker='o', color='green', label='systemA')
ax.scatter(*geometry[1][1:][0], marker='o', color='green', label='systemA')

for x in range(2, len(geometry)):
    ax.scatter(*geometry[x][1:][0], marker='o', color='red', label='systemB')
plt.legend()
plt.show()


<IPython.core.display.Javascript object>

In [6]:
# H_bond = 0.74
# R= 1.2


# # geometry = [f'H 0,0,{-H_bond}; H 0,0,0; H 0,0,{R}; H 0,0,{R+H_bond};']
# # geometry = [
# # ['H@1', (0,0,-H_bond)],
# # ['H@1', (0,0,0)],
# # ['H#2', (0,0,R)],
# # ['H#2', (0,0,R+H_bond)],
# # ]

# CC = 1
# CO = 1
# CH=1
# OH=0.5

# geometry = [
# ['O@1', (-CH*np.sin(2*np.pi/3),CH*np.cos(2*np.pi/3),0)],
# ['H@1', (-CH*np.sin(2*np.pi/3) - OH/2, CH*np.cos(2*np.pi/3)+ OH,0)],
# ['C#1', (0,0,0)],
# ['H#1', (-CH*np.sin(2*np.pi/3),-CH*np.cos(2*np.pi/3),0)],
    
# ['C#2', (CC,0,0)],
# ['H#2', (CC + CH*np.sin(2*np.pi/3),-CH*np.cos(2*np.pi/3),0)],
# ['O#2', (CC + CO*np.sin(2*np.pi/3),CH*np.cos(2*np.pi/3),0)],
# ['H#2', (CC + CO*np.sin(2*np.pi/3)+ OH/2,CH*np.cos(2*np.pi/3) + OH ,0)],
# ]

basis = '6-31g' # 'STO-6G'
low_level_xc_functional_or_HF = 'lda, vwn' #'hf'
high_level_xc_functional = 'b3lyp'

low_level_method = 'rhf'
high_level_ref = 'rhf'
high_level_method = 'mp2'

# Define full molecule

In [7]:
full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol.build()
full_system_mol.atom

[['O@1', (1.2322305822, -0.2731895077, -0.1276123902)],
 ['H@1', (1.2473876659, -0.899873759, 0.615068157)],
 ['C#', (-1.2129704155, -0.2295285634, -0.0097156258)],
 ['H#', (-2.080142536, 0.4329727646, 0.0722817289)],
 ['H#', (-1.2655910941, -0.9539857247, 0.809795344)],
 ['C#', (0.0849758188, 0.5590385475, 0.0510545434)],
 ['H#', (0.1506137362, 1.1200249874, 0.9943015309)],
 ['H#', (0.1316093068, 1.28418054, -0.7645223601)],
 ['H#', (-1.273754156, -0.7748626513, -0.9540587845)]]

In [8]:
full_system_mol.verbose = 1
full_system_mol.max_memory = 8_000 #MB

# Supersystem Calculation

In [9]:
full_system_scf = scf.RKS(full_system_mol)
full_system_scf.verbose=1
full_system_scf.max_memory= 8_000
full_system_scf.conv_tol = 1e-6
full_system_scf.xc = low_level_xc_functional_or_HF
full_system_scf.kernel()

-153.67504736863157

In [10]:
full_system_scf.e_tot

-153.67504736863157

In [11]:
full_system_scf.conv_check

True

In [14]:
two_e_term_total = full_system_scf.get_veff()
e_xc_total = two_e_term_total.exc
v_xc_total = two_e_term_total - full_system_scf.get_j() 

In [ ]:
# full_system_scf.get_fock()
# full_system_scf.get_hcore()

In [ ]:
# full_system_scf.mo_coeff

# Localise orbitals

In [15]:
from pyscf import lo

In [16]:
# C matrix stores the AO to localized orbital coefficients
# C = lo.orth_ao(full_system_sft, 'meta_lowdin')

occ_orbs = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
PM = lo.PipekMezey(full_system_mol, occ_orbs)
PM.pop_method = 'mulliken' # 'meta-lowdin', 'iao', 'becke'
C_mull = PM.kernel()

local_orb = PM.mo_coeff
A_ij = PM.atomic_pops(full_system_mol, local_orb)# full_system_scf.mo_coeff)

In [17]:
np.array_equal(C_mull, local_orb)

True

In [18]:
C_loc_occ = local_orb#[:,full_system_scf.mo_occ>0]

dm_localised = 2* C_loc_occ @ C_loc_occ.conj().T
S = full_system_scf.get_ovlp()

print(np.isclose(np.trace(dm_localised@S), full_system_mol.nelectron))

True


In [ ]:
# S_half = sp.linalg.fractional_matrix_power(S, -0.5)
# C_prime = (S_half@local_orb)[:,full_system_scf.mo_occ>0]
# dm_test = 2* C_prime @ C_prime.conj().T
# print(np.isclose(np.trace(dm_test@S), full_system_mol.nelectron))
# np.trace(dm_test@S)

In [19]:
# https://pubs.acs.org/doi/pdf/10.1021/ct300544e
# paper states there are 5 active orbs... where mulliken_population threshold set to 0.4
# here I get the result if threshold set to 1

AO_slice_matrix = full_system_mol.aoslice_by_atom()
N_active_atoms = 2

active_ao_ind=[]

PS_matrix = dm_localised @ S
atomic_charges_list = full_system_mol.atom_charges()
for atm_i, atm_str in enumerate([atom_str for atom_str, _ in full_system_mol.atom][:N_active_atoms]):
    slice_ind = list(range(AO_slice_matrix[atm_i, 2], AO_slice_matrix[atm_i, 3]))
    
    print(atm_str)
    qA = atomic_charges_list[atm_i] #atomic charge of atom_i
    for mu in slice_ind:
        mulliken_charge = qA - PS_matrix[mu,mu]  # mulliken charge of ao centred on atom_i
        mulliken_population = PS_matrix[mu,mu]  # mulliken pop of ao centred on atom_i
#         print(mulliken_charge)
        print(mulliken_population)
        if mulliken_population>1: ### <--- not sure about this THRESHOLD
            active_ao_ind.append(mu)
        
    print('###')

active_ao_ind

O@1
1.9964612397927226
0.8920838932477323
1.015496003970721
0.968215875631037
0.9448947883441309
0.9918819958417856
0.6088205498099916
0.5352703541202722
0.6125270236954455
###
H@1
0.49968862186149104
0.13815896029013985
###


[0, 2]

In [20]:
# full_system_mol.aoslice_2c_by_atom() # bottom right gives final slice ind

# Given active ao indices, get remaining environment ind

all_ao_indices = set(range(full_system_mol.aoslice_by_atom()[-1,-1]))
enviro_ao_ind= list(all_ao_indices.difference(active_ao_ind))
print(enviro_ao_ind)

[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38]


In [ ]:
########################## mulliken pop check START

In [21]:
# https://github.com/pyscf/pyscf/blob/master/pyscf/scf/hf.py

Mulliken_population = np.einsum('ij,ji->i', dm_localised, S).real
Mulliken_population

array([1.99646124, 0.89208389, 1.015496  , 0.96821588, 0.94489479,
       0.991882  , 0.60882055, 0.53527035, 0.61252702, 0.49968862,
       0.13815896, 1.99563843, 0.65788295, 0.66571022, 0.71391865,
       0.71907163, 0.73257828, 0.29669979, 0.35055016, 0.38020209,
       0.50416687, 0.32993228, 0.50669901, 0.33664599, 1.99553181,
       0.66841821, 0.58595489, 0.63888677, 0.69133959, 0.76644578,
       0.15540664, 0.25395237, 0.37580996, 0.51617413, 0.3335552 ,
       0.51495006, 0.29774741, 0.50342491, 0.30920661])

In [22]:
mulliken_charges = np.zeros(full_system_mol.natm)
for i, s in enumerate(full_system_mol.ao_labels(fmt=None)):
    mulliken_charges[s[0]] += Mulliken_population[i]
mulliken_charges = full_system_mol.atom_charges() - mulliken_charges
mulliken_charges

array([-0.56565172,  0.36215242, -0.5122522 ,  0.16590085,  0.156655  ,
       -0.13174602,  0.15027066,  0.18730253,  0.18736848])

In [23]:
full_system_mol.mulliken_pop(dm=dm_localised)

 ** Mulliken pop  **
pop of  0 O@1 1s        1.99646
pop of  0 O@1 2s        0.89208
pop of  0 O@1 3s        1.01550
pop of  0 O@1 2px       0.96822
pop of  0 O@1 2py       0.94489
pop of  0 O@1 2pz       0.99188
pop of  0 O@1 3px       0.60882
pop of  0 O@1 3py       0.53527
pop of  0 O@1 3pz       0.61253
pop of  1 H@1 1s        0.49969
pop of  1 H@1 2s        0.13816
pop of  2 C# 1s        1.99564
pop of  2 C# 2s        0.65788
pop of  2 C# 3s        0.66571
pop of  2 C# 2px       0.71392
pop of  2 C# 2py       0.71907
pop of  2 C# 2pz       0.73258
pop of  2 C# 3px       0.29670
pop of  2 C# 3py       0.35055
pop of  2 C# 3pz       0.38020
pop of  3 H# 1s        0.50417
pop of  3 H# 2s        0.32993
pop of  4 H# 1s        0.50670
pop of  4 H# 2s        0.33665
pop of  5 C# 1s        1.99553
pop of  5 C# 2s        0.66842
pop of  5 C# 3s        0.58595
pop of  5 C# 2px       0.63889
pop of  5 C# 2py       0.69134
pop of  5 C# 2pz       0.76645
pop of  5 C# 3px       0.15541
pop of 

(array([1.99646124, 0.89208389, 1.015496  , 0.96821588, 0.94489479,
        0.991882  , 0.60882055, 0.53527035, 0.61252702, 0.49968862,
        0.13815896, 1.99563843, 0.65788295, 0.66571022, 0.71391865,
        0.71907163, 0.73257828, 0.29669979, 0.35055016, 0.38020209,
        0.50416687, 0.32993228, 0.50669901, 0.33664599, 1.99553181,
        0.66841821, 0.58595489, 0.63888677, 0.69133959, 0.76644578,
        0.15540664, 0.25395237, 0.37580996, 0.51617413, 0.3335552 ,
        0.51495006, 0.29774741, 0.50342491, 0.30920661]),
 array([-0.56565172,  0.36215242, -0.5122522 ,  0.16590085,  0.156655  ,
        -0.13174602,  0.15027066,  0.18730253,  0.18736848]))

In [ ]:
########################z## mulliken pop check END

In [24]:
print(len(full_system_scf.mo_occ))

occupied_indices = np.where(full_system_scf.mo_occ>0)[0]

active_ind_occ = np.intersect1d(occupied_indices, active_ao_ind)
enviro_ind_occ = np.intersect1d(occupied_indices, enviro_ao_ind)
enviro_ind_occ

39


array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [26]:
active_ind_occ

array([0, 2])

In [25]:
# S_half_neg = sp.linalg.fractional_matrix_power(S, -0.5)
# local_orb_new =S_half_neg @ PM.mo_coeff 

# S_half_pos = sp.linalg.fractional_matrix_power(S, +0.5)
# local_orb_new = S_half_pos @ PM.mo_coeff @ S_half_pos

# local_orb_new = S @ PM.mo_coeff 

local_orb_new = PM.mo_coeff

In [27]:
dm_active = 2* local_orb_new[:, active_ind_occ] @ local_orb_new[:, active_ind_occ].conj().T
dm_enviro = 2* local_orb_new[:, enviro_ind_occ] @ local_orb_new[:, enviro_ind_occ].conj().T

In [28]:
print(f'number of e- in system = {full_system_mol.nelectron}')

print('number of e- in subsystems:', np.trace(dm_active@S) + np.trace(dm_enviro@S))


number of e- in system = 26
number of e- in subsystems: 26.00000000000005


In [29]:
dm_act_and_env = 2* local_orb_new@ local_orb_new.conj().T

np.allclose(dm_active+dm_enviro, dm_act_and_env)

True

# Embedding

In [32]:
#It seems that PySCF lumps J and K in the J array 
J_act = full_system_scf.get_j(dm = dm_active)
K_act = np.zeros_like(J_act)
two_e_term_act =  full_system_scf.get_veff(dm=dm_active)
e_xc_act = two_e_term_act.exc
v_xc_act = two_e_term_act - J_act 

E_act = np.einsum('ij, ij', dm_active, full_system_scf.get_hcore() + J_act/2) + e_xc_act

In [33]:
#It seems that PySCF lumps J and K in the J array 
J_env = full_system_scf.get_j(dm = dm_enviro)
K_env = np.zeros_like(J_env)
two_e_term_env =  full_system_scf.get_veff(dm=dm_enviro)
e_xc_env = two_e_term_env.exc
v_xc_env = two_e_term_env - J_env 

E_env = np.einsum('ij, ij', dm_enviro, full_system_scf.get_hcore() + J_env/2) + e_xc_env

In [34]:
j_cross = 0.5 * (matrix_dot(dm_active, J_env) + matrix_dot(dm_enviro, J_act))
k_cross = 0.0

In [35]:
xc_cross = e_xc_total - e_xc_act - e_xc_env
two_e_cross = j_cross + k_cross + xc_cross

In [58]:
# huzinaga projector 

Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm=dm_act_and_env)
F_gammaB_S = Fock @ dm_enviro @ S
projector = -0.5 * (F_gammaB_S + F_gammaB_S.T)

In [37]:
# mu shift projector

mu = 1e6
projector = mu * (S @ dm_enviro  @ S)

In [59]:
v_emb = (J_env + v_xc_total - v_xc_act + projector)

In [60]:
full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=0,
                       #spin=0,
                      )
full_system_mol_EMBEDDED.build()
full_system_mol_EMBEDDED.nelectron = 2*len(active_ind_occ)

EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
EMBEDDED_full_system_scf.verbose=1
EMBEDDED_full_system_scf.max_memory= 8_000
EMBEDDED_full_system_scf.conv_tol = 1e-6
EMBEDDED_full_system_scf.xc = low_level_xc_functional_or_HF

h_core = EMBEDDED_full_system_scf.get_hcore()
EMBEDDED_full_system_scf.get_hcore = lambda *args: v_emb + h_core
EMBEDDED_full_system_scf.kernel()

10.099141625278278

In [61]:
EMBEDDED_full_system_scf.conv_check

True

In [62]:
EMBEDDED_occ_orbs = EMBEDDED_full_system_scf.mo_coeff[:,EMBEDDED_full_system_scf.mo_occ>0]


density_emb = 2 * EMBEDDED_occ_orbs @ EMBEDDED_occ_orbs.conj().T

J_emb, K_emb =EMBEDDED_full_system_scf.get_jk() 

In [63]:
e_act_emb = matrix_dot(density_emb, h_core + 0.5 * J_emb - 0.25 * K_emb)
e_act_emb

-112.34257479764591

In [64]:
correction = (matrix_dot(v_emb, density_emb - dm_active))
correction

-4.7102027218526953e-07

In [65]:
e_mf_emb = e_act_emb + E_env + two_e_cross + full_system_scf.energy_nuc() + correction
e_mf_emb # <-- energy from embedded DFT calc

-154.4860029905598

In [66]:
full_system_scf.e_tot # <-- energy from normal DFT calc

-153.67504736863157